<a href="https://colab.research.google.com/github/flaviare1s/magazine-capivara/blob/main/Magazine_Capivara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Magazine Capivara

## Imports das Bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Import dos dados

In [2]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vTXDffy5v8dZQdPVO0oZYU-m4DAKYQa5wIO7QvTudUUaDt7gwEInmB5mTmbTxrRjCSqB6zwSr6XoWhA/pub?output=csv')
df.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
0,2023-01-01,19352,6031,13321,76,8,Roupas,1,Oeste
1,2023-01-01,47585,29758,17827,7,13,Móveis,1,Leste
2,2023-01-02,36867,10699,26168,29,12,Eletrônicos,5,Sul
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
4,2023-01-03,8644,4939,3705,33,1,Móveis,3,Norte


## Conexão com o Google Cloud

In [3]:
from google.colab import auth
auth.authenticate_user()
project_id = 'flavia-ad13'
!gcloud config set project {project_id}

from google.cloud import storage
client = storage.Client()
bucket_name = 'capivara-magazine2025'
bucket = client.bucket(bucket_name)

df.to_csv('capivara-bruta.csv', index=False)
destination_blob_name = 'dados-brutos/capivara-bruta.csv'
source_file_name = 'capivara-bruta.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

Updated property [core/project].


## 1. Divisão em Categorias

### Descobrindo quais categorias existem

In [4]:
categorias_unicas = df["Categoria do Produto"].unique()
print(categorias_unicas)

['Roupas' 'Móveis' 'Eletrônicos' 'Brinquedos' 'Alimentos']


### Separando em DataFrames por categoria

In [5]:
dfs_por_categoria = {categoria: df[df["Categoria do Produto"] == categoria] for categoria in df["Categoria do Produto"].unique()}

In [6]:
df_roupas = dfs_por_categoria["Roupas"]
df_moveis = dfs_por_categoria["Móveis"]
df_eletronicos = dfs_por_categoria["Eletrônicos"]
df_brinquedos = dfs_por_categoria["Brinquedos"]
df_alimentos = dfs_por_categoria["Alimentos"]

### Salvando em CSV e enviando para o GCP

In [9]:
df_roupas.to_csv('capivara-bruta-roupas.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-roupas.csv'
source_file_name = 'capivara-bruta-roupas.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [10]:
df_moveis.to_csv('capivara-bruta-moveis.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-moveis.csv'
source_file_name = 'capivara-bruta-moveis.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [11]:
df_eletronicos.to_csv('capivara-bruta-eletronicos.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-eletronicos.csv'
source_file_name = 'capivara-bruta-eletronicos.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [12]:
df_brinquedos.to_csv('capivara-bruta-brinquedos.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-brinquedos.csv'
source_file_name = 'capivara-bruta-brinquedos.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [13]:
df_alimentos.to_csv('capivara-bruta-alimentos.csv', index=False)
destination_blob_name = 'categoria/capivara-bruta-alimentos.csv'
source_file_name = 'capivara-bruta-alimentos.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## 2. Identificação dos produtos mais lucrativos

### Identificando as categorias que geraram mais lucro

In [16]:
lucro_total_por_categoria = df.groupby("Categoria do Produto")["Lucro"].sum().sort_values(ascending=False)
print(lucro_total_por_categoria)

Categoria do Produto
Alimentos      1695177
Brinquedos     1631946
Eletrônicos    1622365
Roupas         1342888
Móveis         1288416
Name: Lucro, dtype: int64


### Calculando a média geral de lucro

In [20]:
media_geral_lucro = df["Lucro"].mean().round(2)
print(media_geral_lucro)

12634.65


### Filtrando os produtos com lucro acima da média

In [22]:
df_lucro_acima_media = df[df["Lucro"] > media_geral_lucro]
df_lucro_acima_media.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
0,2023-01-01,19352,6031,13321,76,8,Roupas,1,Oeste
1,2023-01-01,47585,29758,17827,7,13,Móveis,1,Leste
2,2023-01-02,36867,10699,26168,29,12,Eletrônicos,5,Sul
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
9,2023-01-06,35695,15800,19895,75,2,Alimentos,5,Leste


### Enviando o DataFrame de produtos com lucro acima da média para o bucket no Google Cloud

In [23]:
df_lucro_acima_media.to_csv('produtos-com-lucro-acima-da-media.csv', index=False)
destination_blob_name = 'lucros/produtos-com-lucro-acima-da-media.csv'
source_file_name = 'produtos-com-lucro-acima-da-media.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

## 3. Avaliação da Satisfação do Cliente

### Criando um DataFrame apenas com 'Satisfação do Cliente' nível 5

In [24]:
df_satisfacao_5 = df[df["Satisfação do Cliente"] == 5]
df_satisfacao_5.head()

,Data,Receita,Custo,Lucro,Quantidade de Vendas,Novos Clientes,Categoria do Produto,Satisfação do Cliente,Região
2,2023-01-02,36867,10699,26168,29,12,Eletrônicos,5,Sul
3,2023-01-02,30334,5471,24863,11,24,Brinquedos,5,Norte
7,2023-01-05,43442,32811,10631,32,20,Alimentos,5,Oeste
9,2023-01-06,35695,15800,19895,75,2,Alimentos,5,Leste
10,2023-01-07,2008,1245,763,58,41,Eletrônicos,5,Norte


### Comparando os faturamentos

In [25]:
faturamento_medio_geral = df["Receita"].mean()
print(f"Faturamento Médio Geral: {faturamento_medio_geral:.2f}")

Faturamento Médio Geral: 25647.40


In [26]:
faturamento_medio_satisfacao_5 = df_satisfacao_5["Receita"].mean()
print(f"Faturamento Médio (Satisfação 5): {faturamento_medio_satisfacao_5:.2f}")

Faturamento Médio (Satisfação 5): 26798.24


In [27]:
if faturamento_medio_satisfacao_5 > faturamento_medio_geral:
    print("O faturamento médio das vendas com Satisfação 5 é maior que o faturamento médio geral.")
else:
    print("O faturamento médio das vendas com Satisfação 5 é menor que o faturamento médio geral.")

O faturamento médio das vendas com Satisfação 5 é maior que o faturamento médio geral.


### Enviando para o Google Cloud

In [28]:
df_satisfacao_5.to_csv('satisfacao-maxima.csv', index=False)
destination_blob_name = 'satisfacao/satisfacao-maxima.csv'
source_file_name = 'satisfacao-maxima.csv'
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)